In [2]:
from typing import Dict, List, Callable, Any


class Tool:
    def __init__(
        self,
        name: str,
        description: str,
        input_schema: Dict[str, Any],
        output_schema: Dict[str, Any],
        func: Callable[..., Any],
    ):
        self.name = name
        self.description = description
        self.input_schema = input_schema
        self.output_schema = output_schema
        self.func = func

    def __call__(self, **kwargs):
        return self.func(**kwargs)

In [3]:
from typing import Union, Literal
from pydantic import BaseModel


class ToolRegistry:
    def __init__(self):
        self.tools: Dict[str, Tool] = {}

    def register(self, tool: Tool):
        self.tools[tool.name] = tool

    def get(self, name: str) -> Tool:
        if name not in self.tools.keys():
            raise ValueError(f"Tool '{name}' not found")
        return self.tools[name]

    def list_tools(self) -> List[Dict[str, Any]]:
        return [
            {
                "name": tool.name,
                "description": tool.description,
                "input_schema": tool.input_schema.model_json_schema(),
            }
            for tool in self.tools.values()
        ]

    def get_tool_call_args_type(self) -> Union[BaseModel]:
        input_args_models = [tool.input_schema for tool in self.tools.values()]
        tool_call_args = Union[tuple(input_args_models)]
        return tool_call_args

    def get_tool_names(self) -> Literal[None]:
        return Literal[*self.tools.keys()]

In [7]:
# Example tool definitions

def add(a: int, b: int) -> int:
    return a + b


def multiply(a: int, b: int) -> int:
    return a * b

In [9]:
# Tool input schemas for validation

class ToolAddArgs(BaseModel):
    a: int
    b: int


class ToolMultiplyArgs(BaseModel):
    a: int
    b: int

In [10]:
# Tool registry, create a new tool and add it to this registry then the agent will be able to call it

registry = ToolRegistry()

registry.register(Tool(
    name="add",
    description="Add two numbers",
    input_schema=ToolAddArgs,
    output_schema={"result": "int"},
    func=add,
))

registry.register(Tool(
    name="multiply",
    description="Multiply two numbers",
    input_schema=ToolMultiplyArgs,
    output_schema={"result": "int"},
    func=multiply,
))

In [13]:
# Get type-safe tool names and arguments
ToolNameLiteral = registry.get_tool_names()
ToolArgsUnion = registry.get_tool_call_args_type()

print(f"Available Tools: {ToolNameLiteral}\nTool Arguments: {ToolArgsUnion}")

class ToolCall(BaseModel):
    action: Literal["tool"]
    thought: str
    tool_name: ToolNameLiteral
    args: ToolArgsUnion


class FinalAnswer(BaseModel):
    action: Literal["final"]
    answer: str


LLMResponse = Union[ToolCall, FinalAnswer]

Available Tools: typing.Literal['add', 'multiply']
Tool Arguments: typing.Union[__main__.ToolAddArgs, __main__.ToolMultiplyArgs]


In [15]:
import json
from google import genai
from google.genai import types


class GeminiLLM:
    def __init__(self, client, tool_registry, model="gemini-2.5-flash"):
        self.client = client
        self.model = model
        self.tool_registry = tool_registry
        self.system_instruction = self._create_system_instruction()
        
    def _create_system_instruction(self):
        tools_info = self.tool_registry.list_tools()
        print(tools_info)
        
        system_prompt = f"""
        You are a conversational AI agent that can interact with external tools.
        
        CRITICAL RULES (MUST FOLLOW):
        - You are NOT allowed to perform operations internally that could be performed by an available tool.
        - If a tool exists that can perform any part of the task, you MUST use that tool.
        - You MUST NOT skip tools, even for simple or obvious steps.
        - You MUST NOT combine multiple operations into a single step unless a tool explicitly supports it.
        - You may ONLY produce a final answer when no available tool can further advance the task.
        TOOL USAGE RULES:
        - Each tool call must perform exactly ONE meaningful operation.
        - If the task requires multiple operations, you MUST call tools sequentially.
        - If multiple tools could apply, choose the most specific one.
        RESPONSE FORMAT (STRICT):
        - You MUST respond ONLY in valid JSON.
        - Never include explanations outside JSON.
        - You must choose exactly one action per response.
        
        Available tools:
        {json.dumps(tools_info, indent=2)}
        
        To use a tool, respond with a JSON object with the following structure:
        {{
            "action": "tool",
            "thought": "Your reasoning here",
            "tool_name": "name of the tool to call",
            "inputs": "tool inputs here"
        }}
        
        To give a final answer, respond with a JSON object with the following structure:
        {{
            "action": "final",
            "answer": "your final answer here"
        }}
        """
        return system_prompt